In [1]:
import requests
import gspread
from datetime import date, timedelta, datetime
from time import sleep
#import json
sa = gspread.service_account(filename="sa_creds.json")


In [2]:
#Ryanair queries:
def getDestinations(airportFrom: str):
    base = 'https://www.ryanair.com/api/locate/v5/routes'
    departureAirportCode = f'departureAirportCode={airportFrom.upper()}'
    fields = 'fields=arrivalAirport.code&fields=arrivalAirport.name&fields=arrivalAirport.seoName&fields=arrivalAirport.timeZone&fields=arrivalAirport.city.code&fields=arrivalAirport.city.name&fields=arrivalAirport.coordinates.latitude&fields=arrivalAirport.coordinates.longitude&fields=arrivalAirport.country.code&fields=arrivalAirport.country.name&fields=arrivalAirport.country.currency&fields=operator'
    return f'{base}?{departureAirportCode}&{fields}'

def getAvailabilities(airportFrom: str, airportTo: str):
    base = 'https://www.ryanair.com/api/farfnd/3/oneWayFares'
    return f'{base}/{airportFrom.upper()}/{airportTo.upper()}/availabilities'

def getFares(airportFrom: str, airportTo: str, dateOutIn: str, adult: int=1, teen: int=0, children: int=0, infant: int=0, flexDays: int=6):
    base = 'https://www.ryanair.com/api/booking/v4/en-en/availability'
    
    return f'{base}?ADT={adult}&CHD={children}&Destination={airportTo.upper()}&INF={infant}&Origin={airportFrom.upper()}&TEEN={teen}&IncludeConnectingFlights=false&DateOut={dateOutIn}&FlexDaysOut={flexDays}&DateIn={dateOutIn}&FlexDaysIn={flexDays}&RoundTrip=true&ToUs=AGREED'

def readDestinations(trip, mappingFor, destinationsJSON, destinationsList):
    for d in destinationsJSON:
        dArrivalAirportCode = d['arrivalAirport']['code']
        dAirportName = d['arrivalAirport']['name']
        dAirportSeoName = d['arrivalAirport']['seoName']
        dAirportCountryCode = d['arrivalAirport']['country']['code']
        dAirportCountryName = d['arrivalAirport']['country']['name']
        dAirportCityName = d['arrivalAirport']['city']['name']
        dAirportTimeZone = d['arrivalAirport']['timeZone']
        dCurrency = d['arrivalAirport']['country']['currency']
        dlatitude = str(d['arrivalAirport']['coordinates']['latitude'])
        dlongitude = str(d['arrivalAirport']['coordinates']['longitude'])
        dDateChecked = str(date.today())
        destinationsList.append([trip, mappingFor, dArrivalAirportCode, dAirportName, dAirportSeoName, dAirportCountryCode, dAirportCountryName, dAirportCityName, dAirportTimeZone, dCurrency, dlatitude, dlongitude, dDateChecked])

def buildFareSearch(activeTrips, ryanDestinations):
    for f in activeTrips:
        for i in ryanDestinations:
            if f[0] in i:
                fareDestinations.append([f[0], f[1], f[2], i[1], i[2], f[3]])

rDestinations = []
fareDestinations = []

In [3]:
#getting all worksheets on sheet
wksheets = []
sh = sa.open("Flights")

sh_md = sh.fetch_sheet_metadata()
for s in sh_md['sheets']:
    wksheets.append(s['properties']['title'])
# wksheets

In [4]:
#getting travel route and dates
wks = sh.worksheet('Flights')
flightSearch_all = wks.get_all_values()[1:]

#adding trip to front
for f in flightSearch_all:
    f.insert(0, f'{f[0]}-{f[1]}_{f[2]}')

#getting active trips only
flightSearch_active = [f for f in flightSearch_all if 'TRUE' in f]

#identifying trips that are already in RyanDestinations worksheet
tripsAlreadyIn = []
wks_dest = sh.worksheet('RyanDestinations')
tripsIn = wks_dest.get()
tripsIn = tripsIn[1:]
for f in tripsIn:
    if f[0] not in tripsAlreadyIn:
        tripsAlreadyIn.append(f[0])

#getting list of only new trips
flightSearch_activeNew = [f for f in flightSearch_active if f[0] not in tripsAlreadyIn]

In [5]:
#adding new trips to RyanDestinations worksheet
if len(flightSearch_activeNew) > 0:
    for f in flightSearch_activeNew:
        fromMappingFor = f[1]
        toMappingFor = f[2]
        theTrip = f[0]
        destinations = requests.get(getDestinations(fromMappingFor)).json()
        readDestinations(theTrip, fromMappingFor, destinations, rDestinations)
        sleep(3)
        destinations = requests.get(getDestinations(toMappingFor)).json()
        readDestinations(theTrip, toMappingFor, destinations, rDestinations)
        sleep(4)
    if len(rDestinations) > 0:
        wks_dest.append_rows(rDestinations)

In [17]:
def buildFlightDate(flightDate: str, flightFrom: str, flightTo: str):
    fdate = datetime.strptime(flightDate,"%Y-%m-%d").date()
    if fdate > date.today() + timedelta(days=4):
        fdate = fdate - timedelta(days=3)
    availables = requests.get(getAvailabilities(flightFrom,flightTo)).json()
    for e, a in enumerate(availables):
        if datetime.strptime(a,"%Y-%m-%d").date() < fdate:
            availables[e] = 999
        else:
            availables[e] = abs((datetime.strptime(a,"%Y-%m-%d").date()-fdate).days)
    return fdate + timedelta(days=min(availables))

In [23]:
buildFlightDate('2022-11-30', 'VNO', 'FCO')

datetime.date(2022, 11, 28)

In [62]:




d='2022-11-25'
dt = datetime.strptime(d,"%Y-%m-%d").date()
availables = requests.get(getAvailabilities('VNO','FCO')).json()
if dt > date.today() + timedelta(days=4):
    q = dt - timedelta(days=3)
else:
    q = dt
# print(q)

# dt

for e, a in enumerate(availables):
    availables[e] = datetime.strptime(a,"%Y-%m-%d").date()


for e, a in enumerate(availables):
    availables[e] = abs((a-dt).days)

min(availables)


# datetime.strptime(d,"%Y-%m-%d").date() - timedelta(days=3)

1

In [23]:
fdate = datetime.strptime('2022-11-23',"%Y-%m-%d").date()
availables = requests.get(getAvailabilities('VNO','FCO')).json()
for e, a in enumerate(availables):
    if datetime.strptime(a,"%Y-%m-%d").date() < fdate:
        availables[e] = 9999
    else:
        availables[e] = abs((datetime.strptime(a,"%Y-%m-%d").date()-fdate).days)

min(availables)

0

In [7]:
for f in fareDestinations:
    print(datetime.strptime(f[5],"%Y-%m-%d").date())

In [11]:
flightSearch_active

[['VNO-PSR_2022-12-20', 'VNO', 'PSR', '2022-12-20', 'TRUE']]